In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import requests
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output

In [ ]:
SUGGESTED = 'suggested_is_touched'

In [ ]:
TOLOKA_PATH = 'data/toloka/'

In [ ]:
data = pd.read_csv(os.path.join(TOLOKA_PATH, 'toloka.tsv'), delimiter='\t', index_col='id')
data[SUGGESTED] = str(np.nan)

In [ ]:
row_ids = data.index

def print_img(url, dpi=170):
    image = cv2.imdecode(np.asarray(bytearray(requests.get(url).content), dtype="uint8"), cv2.IMREAD_COLOR)
    plt.imshow(image)
    plt.rcParams['figure.dpi'] = dpi
    plt.show()

def evaluate():
    out_text = widgets.Output(layout={'border': '1px solid black'})
    out_img = widgets.Output(layout={'border': '1px solid black'})
    def update_text(idx):
        with out_text:
            clear_output(wait=True)
            print(f'Current id: {row_ids[idx]}')
            print(f'Is touched: {data.iloc[row_ids[idx]][SUGGESTED]}')
    def display_current(idx):
        update_text(idx)
        with out_img:
            clear_output(wait=True)
            print_img(data.iloc[row_ids[idx]]['image_url'])
            
    idx = 0
    def clicked_prev(_):
        nonlocal idx
        if idx > 0:
            idx -= 1
        display_current(idx)
    def clicked_next(_):
        nonlocal idx
        if idx < len(data)-1:
            idx += 1
        display_current(idx)
    def clicked_touched(_):
        data.at[row_ids[idx], SUGGESTED] = 'touched'
        update_text(idx)
    def clicked_not_touched(_):
        data.at[row_ids[idx], SUGGESTED] = 'not_touched'
        update_text(idx)
    

    button_touched = widgets.Button(description = 'Touched')
    button_touched.on_click(clicked_touched)
    button_not_touched = widgets.Button(description = 'Not Touched')
    button_not_touched.on_click(clicked_not_touched)
    button_prev = widgets.Button(description = 'Previous')
    button_prev.on_click(clicked_prev)
    button_next = widgets.Button(description = 'Next')
    button_next.on_click(clicked_next)
    buttons = widgets.VBox([button_prev, button_touched, button_not_touched, button_next])
    out = widgets.VBox([out_text, out_img])
    box = widgets.HBox([out, buttons])
    display(box)
    display_current(0)

In [ ]:
evaluate()

In [ ]:
assert str(np.nan) not in data[SUGGESTED].unique(), f"{sum(data[SUGGESTED] == str(np.nan))} NaN values still left!"

In [ ]:
data[SUGGESTED].to_csv('suggested.csv')